In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.__version__

'1.4.2'

In [3]:
try:
    df = pd.read_xml("p:/PrintOut_StatReportForExport.xml", xpath="//IHComTable")
except:
    df = pd.read_xml("PrintOut_StatReportForExport.xml", xpath="//IHComTable")

    
now_string = pd.Timestamp.now().strftime('%Y%m%d_%H%M')

try:
    df.to_excel(f"F:/Printer/ALL_{now_string}.xlsx")
except OSError:
    df.to_excel(f"ALL_{now_string}.xlsx")

In [4]:
# qc only
qc_df = df[df['SampleBarcode'].str.contains("QC")]
qc_df = qc_df.copy()
qc_df['BGText'] = qc_df['BGText'].fillna("")
qc_df['BGTextABScrDAT'] = qc_df['BGTextABScrDAT'].fillna("")
# remove not interpretable
qc_df = qc_df[~(qc_df['BGText'].str.contains("not"))]
qc_df = qc_df[~(qc_df['BGTextABScrDAT'].str.contains("not"))]

In [5]:
group = qc_df.groupby(['SampleBarcode','TestDate'])


In [6]:
sample_df_list=[]

for key in group.groups.keys():
    sample_df = group.get_group(key).set_index("WellName")
    sample_result_series = sample_df.FinalResultText
    
    sample_df['ResultText'] = sample_df['BGText']+sample_df['BGTextABScrDAT']
    sample_info_series = sample_df.iloc[0][['TestDate','SampleBarcode','VerifiedByUser', 'ResultText']]

    sample_df_list.append(pd.DataFrame(pd.concat([sample_info_series, sample_result_series])).T)
    

In [7]:
result_df = pd.concat(sample_df_list)
result_df=result_df[~result_df['VerifiedByUser'].isna()]
result_df.fillna("", inplace=True)
result_df = result_df.replace("DP", np.nan).dropna()
result_df.set_index("TestDate", inplace=True)

result_df.sort_index(inplace=True)
result_df.reset_index(inplace=True)

In [8]:


try:
    result_df.to_excel(f"F:/Printer/QC_{now_string}.xlsx")
except OSError:
    result_df.to_excel(f"QC_{now_string}.xlsx")
    

In [9]:
print(result_df)

                          TestDate     SampleBarcode          VerifiedByUser  \
0     2022-04-01T00:37:13.06+08:00  08710691220404QC  automatically accepted   
1    2022-04-01T00:37:37.144+08:00  08720691220404QC                  150948   
2    2022-04-01T00:37:37.221+08:00  08770691220404QC  automatically accepted   
3    2022-04-01T00:37:46.055+08:00  08710691220404QC  automatically accepted   
4    2022-04-01T00:37:46.234+08:00  08720691220404QC                  150948   
..                             ...               ...                     ...   
287  2022-04-30T00:37:58.332+08:00  08720711220502QC  automatically accepted   
288  2022-04-30T00:38:10.292+08:00  08720711220502QC  automatically accepted   
289  2022-04-30T00:38:10.376+08:00  08770711220502QC  automatically accepted   
290  2022-04-30T00:38:19.108+08:00  08710711220502QC  automatically accepted   
291   2022-04-30T00:38:34.08+08:00  08720711220502QC  automatically accepted   

          ResultText Anti-A Anti-B Anti

In [10]:
result_html = result_df.to_html(index=False)

In [11]:
start_date = result_df['TestDate'].apply(pd.to_datetime).min().strftime('%Y/%m/%d')
end_date  = result_df['TestDate'].apply(pd.to_datetime).max().strftime('%Y/%m/%d')

title = f"{start_date}-{end_date}"

In [18]:
html_string = '''

<!DOCTYPE html>
<html>
<head>
<title>{title}</title>

{style}


</head>
<body>

<h1>成大醫院 病理部 血庫組 IH-500 品管報表</h1>

{result}

<footer>
      <p>組長:_________________________</p>
</footer>

</body>
</html>

'''

# print footer
'''
	@media print {
	  footer {
		position: fixed;
		bottom: -15px;
		right: 30px;
		width:100%
		border: 1px solid red;
		font-size: 15px;
		font-family: DFKai-sb;
	  }
       .content-block, p {
		page-break-inside: avoid;
	  }
	}
    
'''
        
style_string='''
<style>
	footer {
	  font-size: 16px;
	  font-family: DFKai-sb;
	  color: black;
	  text-align: right;
	}  
	



	 
    
	table, th, td{
		border: 0;
	}
    
    h1 {
		text-align:center;
		font-family:DFKai-sb;
		font-size: 20px;
	}
	
	.dataframe {
		width: 100%;
		font-family: calibri;
		font-size: 10px;
		border-bottom: 2px solid black;
		border-top: 2px solid black;
	}
	
	.dataframe th{
		border-bottom: 1px solid black;
		text-align: center;
	}
	
	.dataframe td:nth-child(-n+4){
		text-align: left;
	}
	
	.dataframe td{
		text-align: center;
	}
	
	.dataframe tr:nth-child(even){
		background-color: #f2f2f2;
	}
	
</style>
'''

In [19]:
with open("res.html", "w") as f:
     f.write(html_string.format(result=result_html,
                               style=style_string,
                               title=title))
    
